# TEMPLATE 3 - PREDICT

# Libraries

In [1]:
import pandas as pd     # Manejo de dataframes
import numpy as np      # Calculos matriciales
import matplotlib.pyplot as plt    # Visualizacion
import seaborn as sns              # Visualizacion
from google.colab import drive     # Drive en Colab

pd.options.display.max_columns = 100 # para visualizar mas columnas en un df
pd.options.display.max_rows = 100 # para visualizar mas filas en un df
pd.options.display.max_colwidth = None # quitamos el límite del ancho de columnas

import warnings
warnings.filterwarnings("ignore")  # inhabilita warnings

import pickle

In [2]:
from sklearn import set_config
set_config(transform_output = "pandas")

In [3]:
drive.mount('/content/drive') # Nos conectamos a nuestro drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Cambiar por la ruta de la carpeta raíz del proyecto
FOLDER_PATH = '/content/drive/MyDrive/MASTER DATA SCIENCE/ML SUPERVISADO/“DSC{1122} – Entregable SupML - {Blanch, Parals, Ramos, Romans}”'

**NOTA:** Para realizar predicciones a partir de nuevos datos debemos realizar sobre los nuevos datos el mismo preprocesamiento que con los datos de entrenamiento:
* Eliminar mismas variables
* Imputar mismos valores a nulos
* Hacer One-Hot-Encoding
* ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBE repetir:
* Analisis visual de los datos
* Analisis de correlaciones
* Analisis de varianza

# Load clean data

In [5]:
X_pred = pd.read_csv(FOLDER_PATH + '/data/telecom_churn_PREDICT.csv', index_col='Customer_ID')

In [6]:
X_pred.head(3)

,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,change_rev,drop_vce,drop_dat,blck_vce,blck_dat,unan_vce,unan_dat,plcd_vce,plcd_dat,recv_vce,recv_sms,comp_vce,comp_dat,custcare,ccrndmou,cc_mou,inonemin,threeway,mou_cvce,mou_cdat,mou_rvce,owylis_vce,mouowylisv,iwylis_vce,mouiwylisv,peak_vce,peak_dat,mou_peav,mou_pead,opk_vce,opk_dat,mou_opkv,mou_opkd,drop_blk,attempt,complete,callfwdv,callwait,months,uniqsubs,actvsubs,new_cell,crclscod,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays
Customer_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1090001,30.8350,136.75,29.99,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,1.1550,2.0,0.0,10.333333,0.0,13.333333,0.0,66.000000,0.0,3.333333,0.0,40.666667,0.0,1.333333,1.333333,0.896667,0.666667,0.333333,71.746667,0.000000,5.406667,1.333333,0.713333,0.333333,0.473333,30.000000,0.0,54.500000,0.0,13.666667,0.0,22.653333,0.000000,12.333333,66.000000,40.666667,0.0,0.000000,7,1,1,U,AA,N,393,958.0,248.46,182.94,958.0,393,30.49,159.67,65.50,121,52,30,160.0,66.0,31.0,C,ATLANTIC SOUTH AREA,Y,N,149.98999,1.0,1.0,WCMB,0.0,0.0,R,NaN,M,M,1.0,M,3.0,NaN,C,NaN,1.0,U,U,U,U,U,U,Y,216.0
1090002,35.8475,352.75,24.27,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,-35.8475,9.0,0.0,0.000000,0.0,10.000000,0.0,82.666667,1.0,31.000000,0.0,65.333333,1.0,3.666667,9.000000,5.890000,18.333333,1.333333,98.543333,1.023333,83.666667,15.666667,12.683333,1.333333,14.286667,38.000000,0.0,106.926667,0.0,43.666667,1.0,75.283333,1.023333,9.000000,83.666667,66.333333,0.0,1.333333,7,1,1,U,EA,Y,1641,4011.0,391.87,305.59,3998.0,1634,50.93,666.33,272.33,470,181,48,666.0,272.0,51.0,C,NORTHWEST/ROCKY MOUNTAIN AREA,T,N,99.98999,2.0,1.0,WCMB,0.0,0.0,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,0.0,N,U,U,U,U,U,N,101.0
1090003,30.3275,241.50,39.99,0.0000,0.00,0.000,0.0,0.000,0.0000,-86.50,0.8625,2.0,0.0,4.666667,0.0,20.666667,0.0,85.333333,0.0,26.666667,0.0,59.333333,0.0,4.333333,6.333333,4.466667,17.000000,1.666667,143.530000,0.000000,27.373333,9.666667,3.760000,19.666667,21.736667,48.666667,0.0,76.356667,0.0,36.000000,0.0,94.543333,0.000000,6.666667,85.333333,59.333333,0.0,0.333333,9,2,2,Y,A,N,780,1889.0,312.91,219.45,1772.0,734,31.35,253.14,104.86,270,107,30,273.0,109.0,31.0,T,CENTRAL/SOUTH TEXAS AREA,N,N,129.98999,1.0,1.0,WCMB,1.0,0.0,O,13.0,S,M,3.0,M,6.0,1.0,C,A,0.0,N,U,U,U,U,U,Y,262.0


# Load scaler

**NOTA:** si hubiera habido reescalado, hay que utilizar el mismo scaler.transform() que el utilizado para entrenar.

Al haber creado el pipe, lo aplicamos en el dataset de predict y así realizamos exactamente el mismo preprocessig que anteriormente.

In [7]:
# Function Transformer
def replace_negatives(x):
    return np.maximum(x, 0)

In [8]:
preprocess_pipe = pickle.load(open(FOLDER_PATH + '/data/pipeline.pkl', 'rb'))

Realizamos las modificaciones aplicadas al dataset en el preprocessing.

In [9]:
X_pred['has_kids'] = np.where((X_pred.filter(like = 'kid') == 'Y').any(axis=1), 'Y', 'U')

In [10]:
X_pred = preprocess_pipe.transform(X_pred)

Cargamos las features resultantes del preprocessing guardadas anteriormente para tener las mismas columnas en el predict.

In [11]:
features_to_maintain = pickle.load(open(FOLDER_PATH + '/data/features.pkl', 'rb'))

In [12]:
X_pred = X_pred[features_to_maintain]

# Load model

In [13]:
model = pickle.load(open(FOLDER_PATH + '/model/model.pkl', 'rb'))

# Predict

In [14]:
predictions = model.predict(X_pred)

In [15]:
predictions

array([0, 0, 0, ..., 1, 0, 0])

# Save predictions

In [16]:
result = pd.DataFrame(predictions, columns=['churn'], index = X_pred.index)

In [17]:
result

,churn
Customer_ID,
1090001,0
1090002,0
1090003,0
1090004,0
1090005,0
...,...
1099996,1
1099997,1
1099998,1


In [18]:
result.to_csv(FOLDER_PATH + '/predictions.csv')